# Spaceship Titanic: who are the passengers?
Created by Xiaoyu Yang
<br>
Created on 10/15/23

## Step 0: load data from kaggle.com (using kaggle API)

In [3]:
#!kaggle competitions download -c spaceship-titanic

  0%|                                                | 0.00/299k [00:00<?, ?B/s]
100%|████████████████████████████████████████| 299k/299k [00:00<00:00, 4.26MB/s]


In [4]:
#!mv spaceship-titanic.zip data/spaceship-titanic.zip

In [8]:
#from zipfile import ZipFile
#with ZipFile('data/spaceship-titanic.zip', 'r') as files:
#   files.extractall()

## Step 1: let's understand the training data

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib as plt

pd.set_option("display.max_rows", 1000)
pd.set_option("display.max_columns", 1000)

In [2]:
df = pd.read_csv('../data/spaceship_titanic/train.csv')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8693 entries, 0 to 8692
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   8693 non-null   object 
 1   HomePlanet    8492 non-null   object 
 2   CryoSleep     8476 non-null   object 
 3   Cabin         8494 non-null   object 
 4   Destination   8511 non-null   object 
 5   Age           8514 non-null   float64
 6   VIP           8490 non-null   object 
 7   RoomService   8512 non-null   float64
 8   FoodCourt     8510 non-null   float64
 9   ShoppingMall  8485 non-null   float64
 10  Spa           8510 non-null   float64
 11  VRDeck        8505 non-null   float64
 12  Name          8493 non-null   object 
 13  Transported   8693 non-null   bool   
dtypes: bool(1), float64(6), object(7)
memory usage: 891.5+ KB


In [4]:
df.isnull().sum()

PassengerId       0
HomePlanet      201
CryoSleep       217
Cabin           199
Destination     182
Age             179
VIP             203
RoomService     181
FoodCourt       183
ShoppingMall    208
Spa             183
VRDeck          188
Name            200
Transported       0
dtype: int64

In [5]:
df.head(3)

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False


In [ ]:
target_var = 'Transported'
num_vars = ['Age', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']
cat_vars = ['HomePlanet', 'CryoSleep', 'Cabin', 'Destination', 'VIP', 'Name']

In [6]:
df.groupby(['Transported', 'HomePlanet']).PassengerId.nunique()

Transported  HomePlanet
False        Earth         2651
             Europa         727
             Mars           839
True         Earth         1951
             Europa        1404
             Mars           920
Name: PassengerId, dtype: int64

In [7]:
df.groupby(['Transported', 'Destination']).PassengerId.nunique()

Transported  Destination  
False        55 Cancri e       702
             PSO J318.5-22     395
             TRAPPIST-1e      3128
True         55 Cancri e      1098
             PSO J318.5-22     401
             TRAPPIST-1e      2787
Name: PassengerId, dtype: int64